This is a bunch of bits and pieces that will hopefully help you produce outputs for the GSNSW HyMap data.
Personally I dont like notebooks and I am not sure if this will actually be a help or a hindrance to you.

First things first, lets do some library imports

In [1]:
from pathlib import Path

import hdbscan
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.envi as envi
from scipy.interpolate import interp1d
from spectraltools.hulls.convexhulls import uc_hulls
from spectraltools.extraction.extraction import extract_spectral_features

Define the path to the HyMap data. We will then look for all *ref.bil files in that directory and its subdirectories

In [2]:
path = Path(r"Z:\source\GSNSW HyMap Data\AIR_2002_gov_Broken_Hill_HySp_0352\Processed_Raw_Data")
files = [val for val in path.glob(r'**/*ref.bil')]
file_pairs = [(val, Path(str(val).replace('.bil', '.hdr'))) for val in files]

Lets define some products we want. In this case I am only going to consider depth related products. The reason being that they are super simple to define and are fast to produce.
I will do an example of a more comprehensive feature extracted product after. Be warned though that they are not fast due to the size of the spectral imagery. One HyMap image of 512 columns x 8500 rows x 126 bands is roughly the equivelant of 42km of drillcore. Wow!

Additionally, I dont really know how you would make use of multiprocessing in a notebook. In the python file its easy (refer to that if needed)

Just for your interest:
The cell below took 64 minutes to run on my standard laptop. It produced the 5 products for 41 HyMap images.

In [3]:
# define our depth products. We do this by using start and stop wavelength domains. If you use a list of these then you can generate multiple products in a single loop
products = [(550, 750), (700, 1300), (2050, 2150), (2100, 2250), (2210, 2300)]
bands = []
for files in file_pairs:
    # create an output directory
    products_dir = files[0].parent / 'depth_products'
    products_dir.mkdir(parents=True, exist_ok=True)

    # get the image data
    img_data = envi.open(str(files[1]), str(files[0]))
    bands = 1000.0*np.array(img_data.bands.centers)

    for product in products:
        # extract feature/s. Here you really want to set the ordinates_inspection_range to the thing you are after
        features = extract_spectral_features(img_data[:,:,:], bands, do_hull=True, main_guard=False, ordinates_inspection_range=[product[0], product[1]], 
        distance=1, fit_type='crude')

        # now write the product to file
        md = {}
        md['description'] = 'Simple maximum hull quotient depth between ' + str(product[0]) + ' and ' + str(product[1])
        md['lines'] = img_data.metadata['lines']
        md['samples'] = img_data.metadata['samples']
        md['bands'] = '1' #might be 1, might be more. Depending on what you produced.
        md['data type'] = '4' #float32 see https://www.l3harrisgeospatial.com/docs/idl_data_types.html
        md['interleave'] = 'bip'
        band_name = str(product[0]) + '_' + str(product[1])
        md['band names'] = [band_name]
        fileout_name = band_name+'.hdr'
        fileout_name = str(products_dir / fileout_name)
        img_out = envi.create_image(fileout_name, md, force=True)
        img_out_memmap = img_out.open_memmap(writeable=True)
        img_out_memmap = features.extracted_features[:,:,1]

Everything from this point forward is not setup to run through the entire list of HyMap files. However, the approach would be exactly the same as the cell above. You would create a loop that runs over the file_pairs and does various things like the processing you want and then writing outputs to files (dependent on what you want of course).

I have left them as small snippets so as not to confuse the whole notebook.

What if we want to do some hull quotient stuff ourself. This is how you can do it for a single file. Again loop and output as required like the above stuff. However, this is really what the extract_spectral_features with fit_type = 'crude' is doing. Just figured you might want a bit more control.

In [ ]:
# lets just do the first BH file
single_image = envi.open(str(file_pairs[0][1]), str(file_pairs[0][0]))
bands = 1000.0*np.array(single_image.bands.centers)
# let define where we want to look
indices = np.searchsorted(bands, [2100, 2250])
# lets set the data into reflectance between 0-1. You need to directely reference the images like this as the spectral library uses numpy memmaps to reference the data when it is an image file
temp_img = single_image[:,:,:]/10000.0
# get the hull corrected image
hull_corrected_image = uc_hulls(bands[indices[0]:indices[1]+1], temp_img[:,:,indices[0]:indices[1]+1], 0)
# can now find out what the maximum depth etc is. From a memory perspective I would normally just tack the .max(axis=2) onto the previous line
max_depth = hull_corrected_image.max(axis=2)

What if you want to extract some features and do some clustering?
A HUGE HUGE note here!!!
The images in terms of spectral processing really are quite large and are equivelant to tens of km's of drillcore so they take a long time. This probably needs to be done on bowen and it should really be done in a main guard. In the e.g. bit below the process_my_hymap_stuff() would allow one to use the main_guard = True keyword in extract_spectral_features
I am not sure how one implements that on bowen though (Fang should know how). Ultimately some of this stuff is better run not in a notebook but in a scirpt/s 

e.g

def process_my_hymap_stuff():
    # 1) find TSG files as above
    # 2) enter loop for found files
    # 3) extract features where you want the wavelengths, depths and FWHM
    extract_spectral_features(image_data[:,:,:], bands, do_hull=True, max_features = 3, main_guard=True, ordinates_inspection_range=[2100, 2390],
        distance=2, fit_type='cheb', resolution=1)

if __name__ == '__main__":
    process_my_hymap_stuff()

In [ ]:
# first lets open our image
# lets just do the first BH file as previously. I am only reopening them for this cell as a demonstartion. No need to continusouly reopen these
single_image = envi.open(str(file_pairs[0][1]), str(file_pairs[0][0]))
bands = 1000.0*np.array(single_image.bands.centers)
# now lets do a feature extraction. 
features = extract_spectral_features(single_image[:,:,:], bands, do_hull=True, max_features = 2, ordinates_inspection_range=[2100, 2390],
        distance=2, fit_type='cheb', resolution=1, main_guard=False) #note that main_guard is False here. Super slow!!!
# can cluster features if wanted. Need to be warned though this might not scale very well with really large datasets an aletrnative might be needed here
clst = hdbscan.HDBSCAN(min_cluster_size=10, cluster_selection_epsilon=3)
# lets cluster on the 2 deepest wavelengths. The reshape is to flatten the data into a Nxfeatures array since it was originally from the entire image
clst.fit(features.extracted_features[:,:,0,[0, 1]].reshape(-1, 2))
# show me the cluster labels
labels = clst.labels_

plt.scatter(features.extracted_features[:,:,0,0], features.extracted_features[:,:,0,1], c=labels)
plt.show()

What about if you want the GFZ spectral libraries. Well here is how you would grab one of them. Again just stick stuff in a loop if you want to collect them all up at onnce

In [ ]:
# lets get a GFZ spectral library and put it in the same spectral space as the HyMap data. This assumes you have defined the HyMap bands already
path = Path(r"Z:\source\GFZ_Spectral_Libraries\Koerting_GFZ_libs\2019-004_Koerting-et-al_ReMin_REO_version-2.0\hyperspectral-libraries")
files = [r"GFZ_HySpex_REMin", r"GFZ_HySpex_REMin.hdr"]
gfz_data = envi.open(str(path / files[1]), str(path / files[0]))
# lets setup an interpolating function so we can interpolate the spectra to the HyMap wavelengths
f = interp1d(gfz_data.bands.centers, gfz_data.spectra/10000.0, fill_value='extrapolate', kind='cubic')
# convert our spectral library to the HyMap wavelengths (actually just a 1D interpolation). Remember they are reflectance*10000
new_gfz_lib = f(bands) # bands was defined somewhere earlier. If not its the HyMap bands. Both the GFZ and HyMap bands need to be in the same units